<a href="https://colab.research.google.com/github/pemk15/web-crawler/blob/main/TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contrução do Corpus

Aluno: Marcio Vinicius de Souza da Rocha


## Import Section

In [ ]:
from bs4 import BeautifulSoup
import spacy

## Constants Section

In [ ]:
sites = {
    'site1.html': 'https://en.wikipedia.org/wiki/Natural_language_processing'
    , 'site2.html': 'https://cloud.google.com/learn/what-is-natural-language-processing'
    , 'site3.html': 'https://www.techtarget.com/searchenterpriseai/definition/natural-language-processing-NLP'
    , 'site4.html': 'https://www.datarobot.com/blog/what-is-natural-language-processing-introduction-to-nlp/'
    , 'site5.html': 'https://levity.ai/blog/how-natural-language-processing-works'
}

In [ ]:
# some HTML tags that may pass by parser
FORBIDDEN_WORDS = [
    '<br>'
    , '<strong>'
    , '</strong>'
    , '<b>'
    , '</b>'
    , '\xa0'
    , '\n'
    , '--'
    , ' — '
    , '\t'
    , '\xad'
]

## Requests Section

In [ ]:
for site in sites:
  !wget '{sites[site]}' -O '{site}'

--2022-11-12 16:33:47--  https://en.wikipedia.org/wiki/Natural_language_processing
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.153.224, 2620:0:860:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.153.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215645 (211K) [text/html]
Saving to: ‘site1.html’

site1.html          100%[===================>] 210.59K  --.-KB/s    in 0.09s   

2022-11-12 16:33:47 (2.36 MB/s) - ‘site1.html’ saved [215645/215645]

--2022-11-12 16:33:48--  https://cloud.google.com/learn/what-is-natural-language-processing
Resolving cloud.google.com (cloud.google.com)... 74.125.201.101, 74.125.201.100, 74.125.201.102, ...
Connecting to cloud.google.com (cloud.google.com)|74.125.201.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 725021 (708K) [text/html]
Saving to: ‘site2.html’

site2.html          100%[===================>] 708.03K  --.-KB/s    in 0.008s  

2022-11-12 16:33:48 (84.2 MB

## List to String function

Transforms word list to string

In [ ]:
def list_to_string(word_list: list):
  some_string = ''
  for word in word_list:
    if list == type(word):
      word = list_to_string(word)
    if '' != word:
      some_string+='{} '.format(word)
  return some_string

## Format String function

this function receives an string and format then removing multiple white spaces, new lines characters, and mounting the new string.

In [ ]:
def format_string(some_string: str):
  for FWord in FORBIDDEN_WORDS:
    some_string = some_string.replace(FWord,'')
  word_list = some_string.split(' ')
  some_string = list_to_string(word_list)
  return some_string

## Sentences function

this function receives an string and split then into array of sentences, spliting the orginal string by `'.'` and `','`.

replaces all `'.'` by `','` and then splits original string by `','`, then we can get all the sentences of an string.

In [ ]:
#get_sentences = lambda some_string: some_string.replace('.', ',').split(',')
get_sentences = lambda some_string: some_string.split(',')

## Lists to 1D array

this funtion receives an list with n-D and transforms to 1-D list concating the content of chained lists

In [ ]:
def to_1d(lista: list):
  new_list = []
  for obj in lista:
    if list == type(obj):
      new_list += to_1d(obj)
    else:
      new_list.append(obj)
  return new_list

## Web Scrapping for each site that we get

In [ ]:
corpus = {}
nlp = spacy.load('en_core_web_sm')
for site in sites:
  html_doc = open(f'./{site}', 'r')
  soup = BeautifulSoup(html_doc, 'html.parser')
  corpus[site] = to_1d([str(nlp(format_string(a.get_text()))) for a in soup.find_all('p')])

In [ ]:
corpus['site3.html']

['Natural language processing (NLP) is the ability of a computer program to understand human language as it is spoken and written referred to as natural language. It is a component of artificial intelligence (AI). ',
 'NLP has existed for more than 50 years and has roots in the field of linguistics. It has a variety of real-world applications in a number of fields, including medical research, search engines and business intelligence. ',
 'NLP enables computers to understand natural language as humans do. Whether the language is spoken or written, natural language processing uses artificial intelligence to take real-world input, process it, and make sense of it in a way a computer can understand. Just as humans have different sensors such as ears to hear and eyes to see computers have programs to read and microphones to collect audio. And just as humans have a brain to process that input, computers have a program to process their respective inputs. At some point in processing, the input

# Generating Bag of Words

Executando sobre a variável `corpus` e seus indices

```
site1.html
site2.html
site3.html
site4.html
site5.html
```

## Functions

### Is Number

aux function to determine if passed string is a number or not

In [ ]:
def is_number(my_string: str):
  try:
    int(my_string)
    return True
  except:
    return False

### Remove Numbers

function that build an string ingoring numbers from original string

In [ ]:
def remove_numbers(my_string: str):
  aux = ''
  for char in my_string:
    if not is_number(char):
      aux += char
  return aux

### Clean String

function that removes chars and numbers from strings and return the cleaned string


In [ ]:
def clean_string(
    my_string: str
    , f_chars: list = []):
  f_chars+=[',', '(', ')', '{', '}', '[', ']', '"', '+', '!', '@', '#', '$',
            '%', '"', '&', '*', '_', '=', ';', '\\', '|', '<', '>', ':', 'ª', 'º']
  for char in f_chars:
    my_string = my_string.replace(char, '')
  if 'e.g.' != my_string:
    my_string = my_string.replace('.', '')
  return remove_numbers(my_string)


### Bag of Words

function that receives array of phrases then returns the vocabulary of then

In [ ]:
# -------------------------------------------------------------------------
# function that receives array of phrases then returns
# the vocabulary of then
def bag_of_words(phrases: list = []):
  vocabulary = []
  for phrase in phrases:
    for word in phrase.split(' '):
      if (word not in vocabulary) and ('' != word):
        vocabulary.append(clean_string(word))
  return vocabulary


### Sentence Vector

function that receives array of phrases then return array of ocurrences

In [ ]:
# -------------------------------------------------------------------------
# function that receives array of phrases and vocabulary
# then return array of ocurrences
def get_vetor_sentenca(phrases: list, vocabulary: list):
    sentence_vector = [[0 for word in vocabulary] for phrase in phrases]
    for i in range(len(phrases)):
        for j in range(len(vocabulary)):
            sentence_vector[i][j] = phrases[i].count(vocabulary[j])
    return sentence_vector


## Hands On

### Junção de Vetores

unindo todos os vetores em `v`

In [ ]:
v = []
for site in sites:
  v += bag_of_words(corpus[site])

### Criando Vocabulario

para facilitar a criação do vocabulário os conjunto de todos os vetores é transformado em um `DataFrame` (objeto da biblioteca Pandas) e utilizado o método `unique()` do objeto `DataFrame`, este método retorna apenas cada elemento único do conjunto de dados.

In [ ]:
import pandas as pd
df = pd.DataFrame(v)
vocabulario = df[df[0]!=''][0].unique().tolist()
print('Original Lenght: {}\nVocabulary Lenght: {}'.format(
    len(df)
    , len(vocabulario)
))

Original Lenght: 2950
Vocabulary Lenght: 1771


### Aplicação de BoW (Bag of Word) no Corpus

`mat_term_doc` é um dicionário do python contendo como índice o site de referência, cada conteúdo de um índice deste dicionário será a matriz termo documento do site em questão.


In [ ]:
mat_term_doc = {}
for site in sites:
  mat_term_doc[site] = get_vetor_sentenca(corpus[site], vocabulario)

In [ ]:
mat_term_doc['site1.html']

[[1,
  4,
  1,
  1,
  3,
  48,
  1,
  5,
  1,
  4,
  1,
  6,
  1,
  1,
  1,
  2,
  9,
  1,
  1,
  2,
  1,
  13,
  1,
  1,
  2,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  26,
  0,
  0,
  0,
  0,
  15,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
 

# TF-IDF

## Calculando TF

In [ ]:
tf = {}
for site in sites:
  tf[site] = {
      'words': {}
      , 'n_terms': 0
  }
  doc_d = to_1d(corpus[site])
  for phrase in doc_d:
    tf[site]['n_terms'] += len(phrase.split(' '))
    for word in vocabulario:
      word = word.replace(' ', '').replace('”','')
      try:
        tf[site]['words'][word]['ocurrences'] += phrase.count(word)
      except:
        tf[site]['words'][word] = {'ocurrences': phrase.count(word)}
      tf[site]['words'][word]['value'] = tf[site]['words'][word]['ocurrences'] / tf[site]['n_terms']

In [ ]:
site = 'site1.html'
tf[site]['words']['linguistics']

{'ocurrences': 10, 'value': 0.0078125}

## Calculando IDF

In [ ]:
import math

In [ ]:
idf = {}
n_docs = len(sites)
erros = []
for word in vocabulario:
  word = word.replace(' ', '').replace('”','')
  n_docs_cotain_word = 0
  for site in sites:
    doc_d = list_to_string(to_1d(corpus[site]))
    n_docs_cotain_word += 1 if word in doc_d else 0
  try:
    idf[word] = math.log(n_docs/n_docs_cotain_word)
  except:
    erros.append(word)

## Calculando TF-IDF para cada Documento

In [ ]:
tf_idf = {}
for site in sites:
  tf_idf[site] = {}
  for word in vocabulario:
    word = word.replace(' ', '').replace('”','')
    if word not in erros:
      tf_idf[site][word] = tf[site]['words'][word]['value'] * idf[word]

In [ ]:
tf_idf

{'site1.html': {'Natural': 0.0,
  'language': 0.0,
  'processing': 0.0,
  'NLP': 0.0,
  'is': 0.0,
  'a': 0.0,
  'subfield': 0.001257373369089141,
  'of': 0.0,
  'linguistics': 0.0017433089946422638,
  'computer': 0.0010459853967853582,
  'science': 0.0015963300742687211,
  'and': 0.0,
  'artificial': 0.0,
  'intelligence': 0.0005229926983926791,
  'concerned': 0.001257373369089141,
  'with': 0.0,
  'the': 0.0,
  'interactions': 0.0003990825185671803,
  'between': 0.0001743308994642264,
  'computers': 0.0003486617989284528,
  'human': 0.0003486617989284528,
  'in': 0.0,
  'particular': 0.001257373369089141,
  'how': 0.0,
  'to': 0.0,
  'program': 0.0003990825185671803,
  'process': 0.0,
  'analyze': 0.0003486617989284528,
  'large': 0.0031926601485374423,
  'amounts': 0.0007158521342766837,
  'natural': 0.0,
  'data': 0.0,
  'The': 0.0,
  'goal': 0.001257373369089141,
  'capable': 0.001257373369089141,
  'understanding': 0.0,
  'contents': 0.001257373369089141,
  'documents': 0.0015963